In [70]:
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

#試取資料
file = open("/Users/peggy/Documents/109-2(2-2)/machine_learning/MLGame/games/snake/log/snake1.pickle", "rb")
data = pickle.load(file)
file.close()
type(data['ml'])

dict

In [71]:
game_info = data["ml"]["scene_info"]
game_command = data["ml"]["command"]
#print(game_info)
#print(game_command)

In [72]:

for i in range(2,3): 
    path = "/Users/peggy/Documents/109-2(2-2)/machine_learning/MLGame/games/snake/log/snake" + str(i) + ".pickle"
    file = open(path, "rb")
    data = pickle.load(file)
    game_info = game_info + data['ml']['scene_info']
    game_command = game_command + data['ml']['command']
    file.close()  

print(len(game_info))
print(len(game_command))


158034
158034


In [73]:
g = game_info[1]

feature = np.array([g["snake_head"][0], g["snake_head"][1]])
print(feature)
print(game_command[1])
if game_command[1] == "UP": game_command[1] = 0
elif game_command[1] == "DOWN": game_command[1] = 1
elif game_command[1] == "LEFT": game_command[1] = 2
elif game_command[1] == "RIGHT": game_command[1] = 3
else: game_command[1] = 4 

[40 50]
LEFT


In [74]:
for i in range(2, len(game_info) - 1):
    g = game_info[i]
    g_last = game_info[i-1]
    snakehead_x=g["snake_head"][0]
    snakehead_y=g["snake_head"][1]
    x_dir=g['snake_head'][0] - g_last['snake_head'][0]
    y_dir=g['snake_head'][1] - g_last['snake_head'][1]
    food_x=g["food"][0]
    food_y=g["food"][1]
    if x_dir>0 and y_dir == 0:
        direction = 4 
        wall_disx = 300 - g['snake_head'][0]
    elif x_dir<0 and y_dir == 0:
        direction = 3
        wall_disx = g['snake_head'][0]
    elif y_dir>0 and x_dir == 0:
        direction = 2
        wall_disy = 300 - g['snake_head'][1]
    elif y_dir<0 and x_dir == 0:
        direction = 1
        wall_disy = g['snake_head'][1]
        
    if y_dir == 0:
        wall_disy = max(300 - g['snake_head'][1],g['snake_head'][1])
    elif x_dir == 0:
        wall_disx = max(300 - g['snake_head'][0],g['snake_head'][0])
    body = 0
    wall = 0
    #wall            
    #四個角角
    snake_head = g["snake_head"]
    if ((snake_head[0] > 285 and snake_head[1] > 285)or(snake_head[0] > 285 and snake_head[1] < 5) or (snake_head[0] < 5 and snake_head[1] > 285)or(snake_head[0] < 5 and snake_head[1] > 285)):
        #x方向移動
        if direction >= 3:
            if snake_head[0] > 285:
                if snake_head[1] > 285: #右下
                    wall = 1
                if snake_head[1] < 5:#右上
                    wall = 2
            if snake_head[0] < 5:
                if snake_head[1] > 285:#左下
                    wall = 1
                if snake_head[1] < 5:#左上
                    wall = 2
        #y方向移動
        if direction <= 2:
            if snake_head[1] > 285:
                if snake_head[0] > 285: #右下
                    wall = 3
                if snake_head[0] < 5:#左下
                    wall = 4
            if snake_head[1] < 5:
                if snake_head[0] > 285:#右上
                    wall = 3
                if snake_head[0] < 5:#左下
                    wall = 4          
    else:
                #x方向移動
        if direction >= 3:
            if snake_head[0] >= 285:
                if snake_head[1] > 145: #右下
                    wall = 1
                else:#右上
                    wall = 2
            if snake_head[0] < 5:
                if snake_head[1] > 145:#左下
                    wall = 1
                else:#左上
                    wall = 2
                #y方向移動
        if direction <= 2:
            if snake_head[1] >= 285:
                if snake_head[0] > 145: #右下
                    wall = 3
                else:#左下
                    wall = 4
            if snake_head[1] < 5:
                if snake_head[0] > 145:#右上
                    wall = 3
                else:#左上
                    wall = 4


    #是否離body直線距離<=10?body left==3,right==4,up==1,down==2
    for m in range(0,len(g['snake_body'])):
        if direction==3:#left
            if(g['snake_head'][0]==g['snake_body'][m][0]):
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][1]==g['snake_body'][m][1]+10):
                        body = 2
                    elif(g['snake_head'][1]==g['snake_body'][m][1]-10):
                        body = 1
            if (g['snake_head'][0]==g['snake_body'][m][0]+10 and g['snake_head'][1]==g['snake_body'][m][1]): 
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][1]+10==g['snake_body'][m+1][1]):
                        body = 2
                    elif(g['snake_head'][1]-10==g['snake_body'][m+1][1]):
                        body = 1
                        
                    
        elif direction==4:#right 
            if(g['snake_head'][0]==g['snake_body'][m][0]):
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][1]==g['snake_body'][m][1]+10):
                        body = 2
                    elif(g['snake_head'][1]==g['snake_body'][m][1]-10):
                        body = 1
            if (g['snake_head'][0]==g['snake_body'][m][0]-10 and g['snake_head'][1]==g['snake_body'][m][1]): # 
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][1]+10==g['snake_body'][m+1][1]):
                        body = 2
                    elif(g['snake_head'][1]-10==g['snake_body'][m+1][1]):
                        body = 1
                      
                            
        elif direction==1:#up
            if(g['snake_head'][1]==g['snake_body'][m][1]):
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][0]==g['snake_body'][m][0]+10):
                        body = 4
                    elif(g['snake_head'][0]==g['snake_body'][m][0]-10):
                        body = 3
            if (g['snake_head'][1]==g['snake_body'][m][1]+10 and g['snake_head'][0]==g['snake_body'][m][0]): #
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][0]+10==g['snake_body'][m+1][0]):
                        body = 4
                    elif(g['snake_head'][0]-10==g['snake_body'][m+1][0]):
                        body = 3
                        
        elif direction==2:#down
            if(g['snake_head'][1]==g['snake_body'][m][1]):
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][0]==g['snake_body'][m][0]+10):
                        body = 4
                    elif(g['snake_head'][0]==g['snake_body'][m][0]-10):
                        body = 3
            if (g['snake_head'][1]==g['snake_body'][m][1]-10 and g['snake_head'][0]==g['snake_body'][m][0]): # 
                if(m!=len(g['snake_body'])-1):
                    if(g['snake_head'][0]+10==g['snake_body'][m+1][0]):
                        body = 4
                    elif(g['snake_head'][0]-10==g['snake_body'][m+1][0]):
                        body = 3
                        

    
    feature = np.vstack((feature, [g["snake_head"][0], g["snake_head"][1]]))
    if game_command[i] == "UP": game_command[i] = 0
    elif game_command[i] == "DOWN": game_command[i] = 1
    elif game_command[i] == "LEFT": game_command[i] = 2
    elif game_command[i] == "RIGHT": game_command[i] = 3
    else: game_command[i] = 4 
        
answer = np.array(game_command[1:-1])
print(feature)
print(feature.shape)
print(answer)
print(answer.shape)

[[ 40  50]
 [ 30  50]
 [ 20  50]
 ...
 [270   0]
 [280   0]
 [290   0]]
(158032, 2)
[2 2 2 ... 3 3 3]
(158032,)


# KNN


In [77]:
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit

#資料劃分
x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state=9)
#參數區間
param_grid = {'n_neighbors':[1, 2, 3, 4, 5]}
#交叉驗證 
cv = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=12)
grid = GridSearchCV(KNeighborsClassifier(n_neighbors = 5), param_grid, cv=cv, verbose=10, n_jobs=-1) #n_jobs為平行運算的數量
grid.fit(x_train, y_train)
grid_predictions = grid.predict(x_test)

#儲存
file = open('model_KNN.pickle', 'wb')
pickle.dump(grid, file)
file.close()


Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    3.4s remaining:    8.0s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    3.4s remaining:    3.4s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    3.4s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    4.2s finished


In [278]:
#最佳參數
print(grid.best_params_)
#預測結果
#print(grid_predictions)
#混淆矩陣
print(confusion_matrix(y_test, grid_predictions))
#分類結果
print(classification_report(y_test, grid_predictions))

{'n_neighbors': 5}
[[6164   75  373  345    0]
 [  71 6402  356  352    0]
 [ 351  360 8975  105    0]
 [ 368  397  123 8683    0]
 [   5    5    7   25    1]]
              precision    recall  f1-score   support

           0       0.89      0.89      0.89      6957
           1       0.88      0.89      0.89      7181
           2       0.91      0.92      0.91      9791
           3       0.91      0.91      0.91      9571
           4       1.00      0.02      0.05        43

    accuracy                           0.90     33543
   macro avg       0.92      0.72      0.73     33543
weighted avg       0.90      0.90      0.90     33543



# MLPClassifier

In [75]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state = 7)
MLPC = MLPClassifier(random_state=1, max_iter=300).fit(x_train, y_train)

MLPC.predict_proba(x_test)
MLPC.predict(x_test)
MLPC.score(x_test, y_test)

file = open("model_MLPC.pickle", "wb")
pickle.dump(MLPC, file)
file.close()

In [76]:
print("Accuracy score (validation): {0:.3f}".format(MLPC.score(x_test, y_test)))
print("Confusion Matrix for Raandom Forests:")
print(confusion_matrix(y_test, MLPC.predict(x_test)))
print()
print("Classification Report for Random Forests")
print(classification_report(y_test, MLPC.predict(x_test)))

Accuracy score (validation): 0.603
Confusion Matrix for Raandom Forests:
[[ 1575     0     0     0]
 [    0  1009    44   456]
 [    0   849 14806  6395]
 [    0   449 10651 11176]]

Classification Report for Random Forests
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1575
           1       0.44      0.67      0.53      1509
           2       0.58      0.67      0.62     22050
           3       0.62      0.50      0.55     22276

    accuracy                           0.60     47410
   macro avg       0.66      0.71      0.68     47410
weighted avg       0.61      0.60      0.60     47410



# HistGradientBoostingClassifier (會delay不要用)

In [268]:
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier

x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state = 7)
HGBC = HistGradientBoostingClassifier(random_state=1, max_iter=300).fit(x_train, y_train)

HGBC.predict_proba(x_test)
HGBC.predict(x_test)
HGBC.score(x_test, y_test)

file = open("model_HGBC.pickle", "wb")
pickle.dump(HGBC, file)
file.close()

In [269]:
print("Accuracy score (validation): {0:.3f}".format(HGBC.score(x_test, y_test)))
print("Confusion Matrix for Raandom Forests:")
print(confusion_matrix(y_test, HGBC.predict(x_test)))
print()
print("Classification Report for Random Forests")
print(classification_report(y_test, HGBC.predict(x_test)))

Accuracy score (validation): 0.988
Confusion Matrix for Raandom Forests:
[[6812   20   11   14    6]
 [  17 7053    3   19    5]
 [  79   66 9764    2    5]
 [  71   70    5 9476    7]
 [   4    2    2    8   22]]

Classification Report for Random Forests
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      6863
           1       0.98      0.99      0.99      7097
           2       1.00      0.98      0.99      9916
           3       1.00      0.98      0.99      9629
           4       0.49      0.58      0.53        38

    accuracy                           0.99     33543
   macro avg       0.89      0.91      0.90     33543
weighted avg       0.99      0.99      0.99     33543



# DecisionTree

In [270]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state = 7)
DT = DecisionTreeClassifier(random_state=0).fit(x_train, y_train)

DT.predict_proba(x_test)
DT.predict(x_test)
DT.score(x_test, y_test)

file = open("model_DT.pickle", "wb")
pickle.dump(DT, file)
file.close()

In [271]:
print("Accuracy score (validation): {0:.3f}".format(DT.score(x_test, y_test)))
print("Confusion Matrix for Raandom Forests:")
print(confusion_matrix(y_test, DT.predict(x_test)))
print()
print("Classification Report for Random Forests")
print(classification_report(y_test, DT.predict(x_test)))

Accuracy score (validation): 0.989
Confusion Matrix for Raandom Forests:
[[6789   46   15    6    7]
 [  52 7022    6   11    6]
 [  40   29 9826   14    7]
 [  43   33   16 9534    3]
 [   4    2    5    8   19]]

Classification Report for Random Forests
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      6863
           1       0.98      0.99      0.99      7097
           2       1.00      0.99      0.99      9916
           3       1.00      0.99      0.99      9629
           4       0.45      0.50      0.48        38

    accuracy                           0.99     33543
   macro avg       0.88      0.89      0.89     33543
weighted avg       0.99      0.99      0.99     33543



# Extra tree

In [272]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import ExtraTreeClassifier

x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state = 7)
cls = ExtraTreeClassifier(random_state=0)
ET = BaggingClassifier(cls, random_state=0).fit(x_train, y_train)

ET.predict_proba(x_test)
ET.predict(x_test)
ET.score(x_test, y_test)

file = open("model_ET.pickle", "wb")
pickle.dump(ET, file)
file.close()

In [273]:
print("Accuracy score (validation): {0:.3f}".format(ET.score(x_test, y_test)))
print("Confusion Matrix for Raandom Forests:")
print(confusion_matrix(y_test, ET.predict(x_test)))
print()
print("Classification Report for Random Forests")
print(classification_report(y_test, ET.predict(x_test)))

Accuracy score (validation): 0.980
Confusion Matrix for Raandom Forests:
[[6729   32   30   72    0]
 [  30 6962   51   54    0]
 [  76   80 9748   12    0]
 [  78   91   22 9438    0]
 [   4   11    6   12    5]]

Classification Report for Random Forests
              precision    recall  f1-score   support

           0       0.97      0.98      0.98      6863
           1       0.97      0.98      0.98      7097
           2       0.99      0.98      0.99      9916
           3       0.98      0.98      0.98      9629
           4       1.00      0.13      0.23        38

    accuracy                           0.98     33543
   macro avg       0.98      0.81      0.83     33543
weighted avg       0.98      0.98      0.98     33543



# SVM.svc(train很久不要用)

In [274]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SVC = make_pipeline(StandardScaler(), SVC(gamma='auto',probability=True))
SVC.fit(x_train, y_train)

SVC.predict_proba(x_test)
SVC.predict(x_test)
SVC.score(x_test, y_test)

file = open("model_SVC.pickle", "wb")
pickle.dump(SVC, file)
file.close()

In [275]:
print("Accuracy score (validation): {0:.3f}".format(SVC.score(x_test, y_test)))
print("Confusion Matrix for Raandom Forests:")
print(confusion_matrix(y_test, SVC.predict(x_test)))
print()
print("Classification Report for Random Forests")
print(classification_report(y_test, SVC.predict(x_test)))

Accuracy score (validation): 0.981
Confusion Matrix for Raandom Forests:
[[6650   10   56  147    0]
 [   5 6980   48   64    0]
 [  64   59 9792    1    0]
 [  77   62    1 9489    0]
 [   4   12    6   12    4]]

Classification Report for Random Forests
              precision    recall  f1-score   support

           0       0.98      0.97      0.97      6863
           1       0.98      0.98      0.98      7097
           2       0.99      0.99      0.99      9916
           3       0.98      0.99      0.98      9629
           4       1.00      0.11      0.19        38

    accuracy                           0.98     33543
   macro avg       0.98      0.81      0.82     33543
weighted avg       0.98      0.98      0.98     33543

